In [ ]:
# coin modeling


# Coins model
Game parameters to consider
1. bid size
2. win size
3. win probability
4. Available money
5. number of games (try limits)
6. Kelly criterion: https://www.youtube.com/watch?v=-EFyY60UHZ4
o - optimal bid size
m - multiplier in case of win
p - win prob
o = m(p-1)/(m-1)

Visualize a graph % of capital as bid, average gain per bid, one of coin property
x = 1.0
b = 0.2 - bid size
for _ in range(100):
    bx*0.6 - bx*0.4 = 0.2bx

What if I roll a dice. In case of win with p=1/6 it multiplies my bid by 20. What's the average gain? Wha't the average gain per bid?
x - bid size.
20x/6 - 5x/6 = 15x/6
Draw a graph depending on the number of games and different bid strategis (%).
Will I win on average is I bet all my money?
# A coin with 60% win prob doubling the bid

# A bid is constant and declared upfront
2 coins: The first coin takes b1$. In case of win it 
## A bid takes all money in case of loss, but I'm free to choose the bidding size

In [ ]:
# stocks modeling

In [1]:
import numpy as np
import plotly.graph_objects
import ipywidgets
import IPython.display

# Create grid
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)

# Initial function values
Z1 = np.sin(np.sqrt(X**2 + Y**2))  # Function 1 (Scalable)
Z2 = 0.001*(X**2 + Y*X + 1)  # Function 2 (Static)

figure = plotly.graph_objects.FigureWidget([
    plotly.graph_objects.Surface(z=Z1, x=X, y=Y, colorscale="PuOr", opacity=0.7, name="sin(sqrt(x²+y²))"),
    plotly.graph_objects.Surface(z=Z2, x=X, y=Y, colorscale="RdBu", opacity=0.7, name="x² + yx + 1")
], plotly.graph_objects.Layout(scene={'xaxis_title': "X axis", 'yaxis_title': "Y axis", 'zaxis_title': "Z axis"}))

slider = ipywidgets.FloatSlider(
    min=0.1, max=3.0, step=0.1, value=1.0,
)

# Function to update the plot without redrawing
def update_surface(value):
    with figure.batch_update():  # Batch updates to prevent flickering
        figure.data[0].z = value["new"] * np.sin(np.sqrt(X**2 + Y**2))  # Update first function

# Link slider to update function
slider.observe(update_surface, names="value")

# Display slider and figure
IPython.display.display(slider, figure)

FloatSlider(value=1.0, max=3.0, min=0.1)

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(127,59,8)'], [0.1, 'rgb(179,88,6)'], [0.2,
                             'rgb(224,130,20)'], [0.3, 'rgb(253,184,99)'], [0.4,
                             'rgb(254,224,182)'], [0.5, 'rgb(247,247,247)'], [0.6,
                             'rgb(216,218,235)'], [0.7, 'rgb(178,171,210)'], [0.8,
                             'rgb(128,115,172)'], [0.9, 'rgb(84,39,136)'], [1.0,
                             'rgb(45,0,75)']],
              'name': 'sin(sqrt(x²+y²))',
              'opacity': 0.7,
              'type': 'surface',
              'uid': '4a6df587-4e89-4d89-a5f0-d060170ebf49',
              'x': {'bdata': ('AAAAAAAACMDByyl4OQUHwIOXU/ByCg' ... 'PwcgoGQMDLKXg5BQdAAAAAAAAACEA='),
                    'dtype': 'f8',
                    'shape': '50, 50'},
              'y': {'bdata': ('AAAAAAAACMAAAAAAAAAIwAAAAAAAAA' ... 'AAAAAIQAAAAAAAAAhAAAAAAAAACEA='),
                    'dtype': 'f8',
                    'shape': '50, 50'},
       

In [3]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import IPython.display
import threading
import time

x = np.linspace(-3, 3, 50)
b_values = np.linspace(-2, 2, 10)
a_values = np.linspace(0.5, 3.0, 10)
X, B = np.meshgrid(x, b_values)

a_min, a_max = a_values.min(), a_values.max()

# Shared state variables
slider_value = 0.0
user_interacting = False
last_interaction_time = None
resume_requested = False

figure = go.FigureWidget(
    [],
    go.Layout(scene={'xaxis_title': "X axis", 'yaxis_title': "Y axis", 'zaxis_title': "Z axis"})
)

for a in a_values:
    Z = a * X**2 + slider_value * X + B
    surface_color = np.full_like(Z, (a - a_min) / (a_max - a_min))

    figure.add_trace(go.Surface(
        z=Z, x=X, y=B,
        surfacecolor=surface_color,
        colorscale="Hot",
        cmin=0, cmax=1,
        opacity=0.7, showscale=True,
        colorbar_title="a value",
        name=f"a={a:.1f}"
    ))

def update_plot(c):
    """Update the plot with the given slider value."""
    with figure.batch_update():
        for i, a in enumerate(a_values):
            Z_new = a * X**2 + c * X + B
            surface_color = np.full_like(Z_new, (a - a_min) / (a_max - a_min))
            figure.data[i].z = Z_new
            figure.data[i].surfacecolor = surface_color

def on_slider_change(change):
    """Detects when the slider is moved."""
    global slider_value, user_interacting, last_interaction_time, resume_requested
    slider_value = change["new"]
    last_interaction_time = time.time()
    user_interacting = True  # Pause animation immediately
    resume_requested = False  # Prevent unwanted jumps
    update_plot(slider_value)

def debounce_resume():
    """Waits for 1.5s after the last interaction before resuming animation."""
    global user_interacting, last_interaction_time, resume_requested
    while True:
        if user_interacting and last_interaction_time:
            elapsed = time.time() - last_interaction_time
            if elapsed > 1.5:
                user_interacting = False
                resume_requested = True  # Allow animation to continue
        time.sleep(0.1)

def animate_slider():
    """Continuously animate the slider unless the user is interacting."""
    global slider_value, user_interacting, resume_requested
    while True:
        if not user_interacting and resume_requested:
            resume_requested = False  # Ensure animation resumes smoothly
            time.sleep(0.1)

        if not user_interacting:
            slider_value += 0.1
            if slider_value > 5:
                slider_value = -5
            slider.value = slider_value  # Update without overriding manual input
        time.sleep(0.1)

# Create slider
slider = widgets.FloatSlider(value=slider_value, min=-5, max=5, step=0.1)
slider.observe(on_slider_change, names="value")

# Display slider and figure
IPython.display.display(slider, figure)

# Run animation and debounce mechanism in the background using threads
# threading.Thread(target=animate_slider, daemon=True).start()
# threading.Thread(target=debounce_resume, daemon=True).start()


FloatSlider(value=0.0, max=5.0, min=-5.0)

FigureWidget({
    'data': [{'cmax': 1,
              'cmin': 0,
              'colorbar': {'title': {'text': 'a value'}},
              'colorscale': [[0.0, 'rgb(0,0,0)'], [0.3333333333333333,
                             'rgb(230,0,0)'], [0.6666666666666666,
                             'rgb(255,210,0)'], [1.0, 'rgb(255,255,255)']],
              'name': 'a=0.5',
              'opacity': 0.7,
              'showscale': True,
              'surfacecolor': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAA=='),
                               'dtype': 'f8',
                               'shape': '10, 50'},
              'type': 'surface',
              'uid': '1ccf9f37-e69a-422c-95f7-6d4e2be6dbe7',
              'x': {'bdata': ('AAAAAAAACMDByyl4OQUHwIOXU/ByCg' ... 'ByCgZAwMspeDkFB0AAAAAAAAAIQA=='),
                    'dtype': 'f8',
                    'shape': '10, 50'},
              'y': {'bdata': ('AAAAAAAAAMAAAAAAAAAAwAAAAAAAAA' ... 'AAAABAAAAAAAAAAEAAAA

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import plotly.io as pio
import threading
import time

x = np.linspace(-3, 3, 50)
b_values = np.linspace(-2, 2, 10)
a_values = np.linspace(0.5, 3.0, 10)
X, B = np.meshgrid(x, b_values)

a_min, a_max = a_values.min(), a_values.max()

# Shared state variables
slider_value = 0.0
user_interacting = False
last_interaction_time = None
resume_requested = False

figure = go.Figure(
    layout=go.Layout(scene={'xaxis_title': "X axis", 'yaxis_title': "Y axis", 'zaxis_title': "Z axis"})
)

for a in a_values:
    Z = a * X**2 + slider_value * X + B
    surface_color = np.full_like(Z, (a - a_min) / (a_max - a_min))

    figure.add_trace(go.Surface(
        z=Z, x=X, y=B,
        surfacecolor=surface_color,
        colorscale="Hot",
        cmin=0, cmax=1,
        opacity=0.7, showscale=True,
        colorbar_title="a value",
        name=f"a={a:.1f}"
    ))

def update_plot(c):
    """Update the plot with the given slider value."""
    with figure.batch_update():
        for i, a in enumerate(a_values):
            Z_new = a * X**2 + c * X + B
            surface_color = np.full_like(Z_new, (a - a_min) / (a_max - a_min))
            figure.data[i].z = Z_new
            figure.data[i].surfacecolor = surface_color

def on_slider_change(change):
    """Detects when the slider is moved."""
    global slider_value, user_interacting, last_interaction_time, resume_requested
    slider_value = change["new"]
    last_interaction_time = time.time()
    user_interacting = True  # Pause animation immediately
    resume_requested = False  # Prevent unwanted jumps
    update_plot(slider_value)

def debounce_resume():
    """Waits for 1.5s after the last interaction before resuming animation."""
    global user_interacting, last_interaction_time, resume_requested
    while True:
        if user_interacting and last_interaction_time:
            elapsed = time.time() - last_interaction_time
            if elapsed > 1.5:
                user_interacting = False
                resume_requested = True  # Allow animation to continue
        time.sleep(0.1)

def animate_slider():
    """Continuously animate the slider unless the user is interacting."""
    global slider_value, user_interacting, resume_requested
    while True:
        if not user_interacting and resume_requested:
            resume_requested = False  # Ensure animation resumes smoothly
            time.sleep(0.1)

        if not user_interacting:
            slider_value += 0.1
            if slider_value > 5:
                slider_value = -5
            slider.value = slider_value  # Update without overriding manual input
        time.sleep(0.1)

# Create slider
slider = widgets.FloatSlider(value=slider_value, min=-5, max=5, step=0.1)
slider.observe(on_slider_change, names="value")

# Display slider and figure
widgets.VBox([slider])

# Show figure in a separate window
pio.show(figure)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D

# Create a simple matplotlib figure
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(np.sqrt(X**2 + Y**2))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.subplots_adjust(left=0.1, bottom=0.25)
surface = ax.plot_surface(X, Y, Z, cmap='viridis')
colorbar = fig.colorbar(surface, ax=ax, shrink=0.5, aspect=5)

# Create a slider axis and slider
ax_slider = plt.axes([0.1, 0.05, 0.8, 0.05], facecolor='lightgoldenrodyellow')
slider = Slider(ax_slider, 'C', -5.0, 5.0, valinit=0)

# Initial scatter plot
scatter = ax.scatter([], [], [], color='r')

def update(val):
    """Update the plot with the given slider value."""
    c = slider.val
    Z_new = np.sin(np.sqrt(X**2 + Y**2)) + c
    surface.remove()
    surface = ax.plot_surface(X, Y, Z_new, cmap='viridis')
    
    # Update scatter plot
    scatter._offsets3d = (X.ravel(), Y.ravel(), (Z_new + c).ravel())
    
    fig.canvas.draw_idle()

# Update the plot when the slider value changes
slider.on_changed(update)

# Create tkinter window
root = tk.Tk()
root.title("Interactive Matplotlib Graph")
root.geometry("800x600")

# Embed the matplotlib figure in the tkinter window
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

root.mainloop()

In [11]:
import numpy as np
import ipywidgets as widgets

import plotly.graph_objects as go
import IPython.display

# Create grid
x = np.linspace(-3, 3, 50)
real_values = np.linspace(-2, 2, 10)
imag_values = np.linspace(-2, 2, 10)
X, R, I = np.meshgrid(x, real_values, imag_values)

# Initial function values
a = 1.0
B = R + 1j * I
Z = a * X**2 + B * X

# Normalize Z values to a fixed range [0, 1]
Z_min, Z_max = Z.real.min(), Z.real.max()
Z_normalized = (Z.real - Z_min) / (Z_max - Z_min)

# Define color schemes
color_schemes = ["Viridis", "Cividis", "Plasma", "Inferno", "Magma", "Turbo", "Jet", "Rainbow", "Portland", "Blackbody"]

# Create figure
figure = go.FigureWidget(
    layout=go.Layout(scene={'xaxis_title': "X axis", 'yaxis_title': "Real part of B", 'zaxis_title': "Imaginary part of B"})
)

# Add surfaces for different complex B values
for i, (r, im) in enumerate(zip(real_values, imag_values)):
    B = r + 1j * im
    Z = a * X**2 + B * X
    Z_normalized = (Z.real - Z_min) / (Z_max - Z_min)
    surface_color = np.full_like(Z.real, i / len(real_values))
    figure.add_trace(go.Surface(
        z=Z_normalized, x=X[:, :, 0], y=R[:, :, 0],
        surfacecolor=surface_color[:, :, 0],
        colorscale=color_schemes[i % len(color_schemes)],
        cmin=0, cmax=1,
        opacity=0.5, showscale=False,
        name=f"B={r:.1f}+{im:.1f}j"
    ))

# Add semi-transparent planes for X-Y, X-Z, and Y-Z
# X-Y plane
figure.add_trace(go.Surface(
    z=np.zeros_like(X[:, :, 0]), x=X[:, :, 0], y=R[:, :, 0],
    colorscale=[[0, 'rgba(0,0,0,0.1)'], [1, 'rgba(0,0,0,0.1)']],
    opacity=0.3, showscale=False,
    name="X-Y plane"
))

# X-Z plane
figure.add_trace(go.Surface(
    z=Z_normalized[:, :, 0], x=X[:, :, 0], y=np.zeros_like(R[:, :, 0]),
    colorscale=[[0, 'rgba(0,0,0,0.1)'], [1, 'rgba(0,0,0,0.1)']],
    opacity=0.3, showscale=False,
    name="X-Z plane"
))

# Y-Z plane
figure.add_trace(go.Surface(
    z=Z_normalized[:, :, 0], x=np.zeros_like(X[:, :, 0]), y=R[:, :, 0],
    colorscale=[[0, 'rgba(0,0,0,0.1)'], [1, 'rgba(0,0,0,0.1)']],
    opacity=0.3, showscale=False,
    name="Y-Z plane"
))

# Function to update the plot
def update_plot(a):
    with figure.batch_update():
        for i, (r, im) in enumerate(zip(real_values, imag_values)):
            B = r + 1j * im
            Z = a * X**2 + B * X
            Z_normalized = (Z.real - Z_min) / (Z_max - Z_min)
            surface_color = np.full_like(Z.real, i / len(real_values))
            figure.data[i].z = Z_normalized[:, :, 0]
            figure.data[i].surfacecolor = surface_color[:, :, 0]

# Create slider for 'a' value
a_slider = widgets.FloatSlider(value=a, min=0.1, max=3.0, step=0.1, description='a value')
widgets.interactive(update_plot, a=a_slider)

# Display slider and figure
IPython.display.display(a_slider, figure)

FloatSlider(value=1.0, description='a value', max=3.0, min=0.1)

FigureWidget({
    'data': [{'cmax': 1,
              'cmin': 0,
              'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58'],
                             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
              'name': 'B=-2.0+-2.0j',
              'opacity': 0.5,
              'showscale': False,
              'surfacecolor': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAA=='),
                               'dtype': 'f8',
                               'shape': '10, 50'},
              'type': 'surface',
              'uid': 'de6055ea-4f8d-4869-bbfd-33dd8f8ad318',
              'x': {'bdata': ('AAAAAAAACMDByyl4OQUHwIO